In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

dataFile = pd.read_csv('./Mixcancer.csv')
x = dataFile.values[:, 1:]
y = dataFile.values[:, 0]
y = y.reshape(len(y),1)
x = preprocessing.MinMaxScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0, shuffle=True)

In [2]:
def sigmoid(v):
    return 1/(1+np.exp(-v))

def sigmoid_der(v):
    return sigmoid(v)*(1-sigmoid(v))

def reLu(v):
    return np.maximum(v,0)

def reLu_der(v):
    v[v<=0] = 0
    v[v>0] = 1
    return v

In [3]:
def crossEntrop(o,y):
    return (-y*(np.log(o)) - (1-y)* np.log(1-o))

def crossEntrDeriv(o,y):
    return -(y/o - (1-y)/(1-o))

In [4]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True)
np.random.seed(0)

l = 0.01
epochs = 2500
batch_size = 8

trainError = []
testError = []
trainAcc = []
testAcc = []
fMeasure = []

for train_index, test_index in kf.split(x):
    w1 = np.random.uniform(-1,1,[len(x[0]),40]) #weights of the first layer
    b1 = np.zeros([1,40]) #bias of the first layer
    w2 = np.random.uniform(-1,1,[40,1]) #weights of the second layer
    b2 = 0 #bias of the second layer
    
    for epoch in range(epochs):
        for i in range(0, x[train_index].shape[0], batch_size):
            x_mini = x[train_index][i:i + batch_size-1, :]
            y_mini = y[train_index][i:i + batch_size-1, :]

            #feedforward
            in1 = x_mini@w1 + b1
            o1 = reLu(in1)
            in2 = o1@w2 + b2
            o2 = sigmoid(in2)

            #error calculation
            error = crossEntrop(o2 ,y_mini).mean()

            #backpropagation Layer 2
            dE_dO2 = crossEntrDeriv(o2, y_mini)
            dO2_dIn2 = sigmoid_der(in2)
            dIn2_dW2 = o1
            dIn2_B2 = 1
            dE_dW2 = (1/x_mini.shape[0])*dIn2_dW2.T@(dE_dO2*dO2_dIn2)
            dE_dB2 = (1/x_mini.shape[0])*np.ones([1,len(x_mini)])@(dE_dO2*dO2_dIn2)

            #backpropagation Layer 1
            dIn2_dO1 = w2
            dO1_dIn1 = reLu_der(in1)
            dIn1_dW1 = x_mini
            dE_dW1 = (1/x_mini.shape[0])*dIn1_dW1.T@((dE_dO2*dO2_dIn2@dIn2_dO1.T)*dO1_dIn1)
            dE_dB1 = (1/x_mini.shape[0])*np.ones([len(x_mini)])@((dE_dO2*dO2_dIn2@dIn2_dO1.T)*dO1_dIn1)

            #updating parameters
            b2-=l*dE_dB2
            w2-=l*dE_dW2
            b1-=l*dE_dB1
            w1-=l*dE_dW1
        
        if (epoch % 100 == 0):
           print('Loss: ' + str(crossEntrop(sigmoid(reLu(x_test@w1+b1)@w2+b2), y_test).mean()))
        
        #Accuracy
        test_preds = np.where(sigmoid(reLu(x[test_index]@w1+b1)@w2+b2) > 0.5,1,0)
        testAcc.append(metrics.accuracy_score(y[test_index], test_preds))
        
    test_preds = np.where(sigmoid(reLu(x[test_index]@w1+b1)@w2+b2) > 0.5,1,0)
    fMeasure.append(metrics.f1_score(test_preds, y[test_index]))
    print('F-Measure: ' + str(fMeasure[-1]))
    
print('Final Accuracy: ' + str(testAcc[-1]))
print('Average Accuracy: ' + str(np.mean(testAcc)))
print('Average F-Meaure: ' + str(np.mean(fMeasure)))


Loss: 0.8421333398124283
Loss: 0.16172803319283913
Loss: 0.14020356835806155
Loss: 0.13153396487041016
Loss: 0.1264504848467662
Loss: 0.1225861457166549
Loss: 0.1189407193142365
Loss: 0.11566587865000434
Loss: 0.11292275661894993
Loss: 0.11009375106043107
Loss: 0.1075911639825442
Loss: 0.1050815700867344
Loss: 0.10301262028734788
Loss: 0.10099226516752154
Loss: 0.09919750730703725
Loss: 0.09726881700079033
Loss: 0.09550559103223716
Loss: 0.09392159369340596
Loss: 0.09241551325051345
Loss: 0.09090492027215617
Loss: 0.08956765221109862
Loss: 0.0881153577626935
Loss: 0.0866860040272183
Loss: 0.08553401638871773
Loss: 0.08443638042678957
F-Measure: 0.967741935483871
Loss: 0.6754007865291582
Loss: 0.14447276979498366
Loss: 0.1344755898265812
Loss: 0.12806171310076694
Loss: 0.1224009309208656
Loss: 0.11720529464031205
Loss: 0.11253082277903266
Loss: 0.10834130373509188
Loss: 0.10452819284797862
Loss: 0.10112453976730651
Loss: 0.0981127884392562
Loss: 0.09580982516112037
Loss: 0.0935804616315